In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='tanh'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
class LRReducer(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):
    old_lr = self.model.optimizer.lr.read_value()
    new_lr = old_lr * 0.998
    #print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
    self.model.optimizer.lr.assign(new_lr)

In [ ]:
model.fit(x_train, y_train,batch_size=512, epochs=100,verbose=0,validation_data=(x_test,y_test),callbacks=[LRReducer()])

In [ ]:
model.evaluate(x_test,  y_test, verbose=1, batch_size=8192)

In [ ]:
for i in range(3):
    plt.imshow(x_test[i],cmap='gray')
    plt.show()
    print("Prediction:" + np.array2string(np.argmax(model.predict(np.array([x_test[i]])))) + "  Target:" + np.array2string(y_test[i]))

In [ ]:
imdb=ks.datasets.imdb

In [ ]:
(x2_train,y2_train),(x2_test,y2_test)=imdb.load_data(num_words=100000)
x2_train=ks.preprocessing.sequence.pad_sequences(x2_train,value=0,padding="post",maxlen=256)
x2_test=ks.preprocessing.sequence.pad_sequences(x2_test,value=0,padding="post",maxlen=256)

In [ ]:
index=dict([(a,b+3) for (a,b) in imdb.get_word_index().items()])
index["<PAD>"]=0
index["^"]=1
index["<UNK>"]=2
index["<UNUSED>"]=3
index_flip=dict([(b,a) for (a,b) in index.items()])

In [ ]:
def decode(code):
    return " ".join(index_flip.get(x,"#") for x in code)
def encode(setence):
    return np.array([index.get(x,2) for x in setence.split()])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="Z:\\log", histogram_freq=1)

In [ ]:
loss_fn2 = tf.keras.losses.BinaryCrossentropy()

model2 = tf.keras.models.Sequential([
    ks.layers.Embedding(100000,16),
    ks.layers.GlobalAveragePooling1D(),
    ks.layers.Dense(64, activation='selu'),
    ks.layers.Dropout(0.2),
    ks.layers.Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [ ]:
model2.fit(x2_train, y2_train,batch_size=5000, epochs=100,verbose=0,validation_data=(x2_test,y2_test),callbacks=[LRReducer(),tensorboard_callback])

In [ ]:
model2.predict(np.array([encode("dont waste your time on this")]))[0,0]

In [ ]:
model2.predict(np.array([encode("thumbs up")]))[0,0]

In [ ]:
model2.predict(np.array([encode("dont waste your time on this")]))[0,0]